In [1]:
import json
import os

In [3]:
result_dir = "/data/home/keminglu/workspace/transformers-bloom-inference/results"
task = "relation_extraction"
dataset = "redocred"
data_file_path = os.path.join(result_dir, f"results_{task}_{dataset}.json")

In [4]:
results = json.load(open(data_file_path))

In [9]:
pos_true = 0
n_pred = 0
n_true = 0
for pred, true in zip(results["preds"], results["trues"]):
    pred_flat = set()
    for h, t, rels in pred:
        for r in rels:
            pred_flat.add((h, t, r))
    true_flat = set()
    for h, t, r in true:
        true_flat.add((h, t, r))
    
    pos_true += len(true_flat.intersection(pred_flat))
    n_pred += len(pred_flat)
    n_true += len(true_flat)

precision = pos_true / n_pred
recall = pos_true / n_true
f1 = 2 * precision * recall / (precision + recall)
print(f"precision: {precision}, recall: {recall}, f1: {f1}")

precision: 0.31443415405466457, recall: 0.08272910971116129, f1: 0.13099327153813578


In [10]:
dataset = "retacred"
data_file_path = os.path.join(result_dir, f"results_{task}_{dataset}.json")

In [11]:
results = json.load(open(data_file_path))

In [ ]:
preds = []
for pred in results["preds"]:
    if len(pred) == 0:
        pred = "no_relation"
    else:
        pred = pred[0][2][0]
    preds.append(pred)

## Wiki Eval

In [10]:
data_file = "/data/home/keminglu/workspace/evaluation_corpus/wiki_eval/finetune_1b1_data_v2_epoch_1_corpus_filtered_output.txt"

In [11]:
data = [json.loads(line) for line in open(data_file).readlines()]

In [5]:
ood_recall = 0
ood_correct_title = 0
ood_mention_num = 0
id_recall = 0
id_correct_title = 0
id_mention_num = 0
for sample in data:
    targets = sample['targets']
    if "prompt" in sample:
        outputs = json.loads(sample["prompt"] + " " + sample['output'])
    else:
        outputs = json.loads(sample['output'])


    ood_mentions = {each['mention']: each['title'] for each in targets["entities"] if each['ood'] != 'in'}
    ood_mention_num += len(ood_mentions)
    id_mentions = {each['mention']: each['title'] for each in targets["entities"] if each['ood'] == 'in'}
    id_mention_num += len(id_mentions)
    for each in outputs["entities"]:
        if each['mention'] in ood_mentions:
            ood_recall += 1
            if each['title'] == ood_mentions[each['mention']]:
                ood_correct_title += 1
            else:
                print(each, ood_mentions[each['mention']])
        elif each['mention'] in id_mentions:
            id_recall += 1
            if each['title'] == id_mentions[each['mention']]:
                id_correct_title += 1

print(ood_recall/ood_mention_num, ood_correct_title/ood_mention_num, ood_mention_num)
print(id_recall/id_mention_num, id_correct_title/id_mention_num, id_mention_num)

{'mention': 'Slow Burn', 'title': 'Slow Burn (album)', 'description': {'english': 'album by T. G. Sheppard'}, 'type': ['Album']} Slow Burn (T. G. Sheppard album)
{'mention': 'Too Good to Stop Now', 'title': 'Too Good to Stop Now', 'description': {'english': 'album by John Schneider'}, 'type': ['Album']} Too Good to Stop Now (John Schneider album)
{'mention': 'Montana', 'title': '2010–11 Montana Grizzlies basketball team', 'description': {'english': 'American college basketball season'}} 2009–10 Montana Grizzlies basketball team
{'mention': '13 Paoni', 'title': 'Paoni 13 (Coptic Orthodox liturgics)', 'description': {'english': 'eleventh day of the tenth Coptic month'}} Paoni 13
{'mention': 'Government of Moldova', 'title': 'Government of Moldova'} Cabinet of Moldova
{'mention': 'Morristown Historic District', 'title': 'Morristown Historic District (Morristown, New Jersey)', 'description': {'english': 'historic district in Morristown, New Jersey'}, 'type': ['Historic district']} Morristo

In [26]:
ood_mentions = []
for sample in data:
    ood = [each for each in sample['targets']['entities'] if each['ood'] != 'in']
    for each in ood:
        ood_mentions.append({
            "input_text": sample['inputs'],
            "prompt": '{"mention": "%s", "title": "%s", "description":' % (each['mention'], each['title']),
            "true": each['description'] if 'description' in each else None
        })

with open("/data/home/keminglu/workspace/evaluation_corpus/wiki_eval/ood_ent_description_eval.jsonl", "w") as f:
    for record in ood_mentions:
        f.write(json.dumps(record) + "\n")

In [15]:
output = json.loads(data[0]['output'])
entities = data[0]['targets']['entities']
triplets = output['triplets']

ood_ent_fn = lambda mention: [each['ood'] != "in" for each in entities if each['mention'] == mention][0]
ood_triplets = []
for t in triplets:
    if ood_ent_fn(t["head"]) or ood_ent_fn(t["tail"]):
        ood_triplets.append(t)

In [16]:
ood_triplets

[{'head': 'Altiri Chiba',
  'tail': 'basketball',
  'relations': [{'title': 'sport',
    'description': {'english': 'sport that the subject participates or participated in or is associated with'},
    'aliases': {'english': ['sports', 'sport played', 'play', 'plays']}}]}]